<a href="https://colab.research.google.com/github/roxacarv/sistemaderecomendacao_mal/blob/master/Dados_MAL_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Recomendação de Animes usando KNN

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importando bibliotecas necessárias

In [52]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

##Importando dataset de animes

In [53]:
# Importando os dados relacionados aos animes
animes = pd.read_csv("drive/MyDrive/animal_mal/anime.csv", encoding="latin-1")

In [55]:
animes=animes[["MAL_ID", "Name", "Genres", "English name", "Type", "Rating"]]
animes

,MAL_ID,Name,Genres,English name,Type,Rating
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,TV,R - 17+ (violence & profanity)
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,Movie,R - 17+ (violence & profanity)
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,TV,PG-13 - Teens 13 or older
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,TV,PG-13 - Teens 13 or older
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,TV,PG - Children
...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,"Adventure, Mystery, Supernatural",Unknown,ONA,Unknown
17558,48483,Mieruko-chan,"Comedy, Horror, Supernatural",Unknown,TV,Unknown
17559,48488,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry â SOTSU,TV,R - 17+ (violence & profanity)
17560,48491,Yama no Susume: Next Summit,"Adventure, Slice of Life, Comedy",Unknown,TV,PG-13 - Teens 13 or older


## Importando dataset de perfis de usuários

In [56]:
# Importando os dados de ratings
ratings = pd.read_csv("drive/MyDrive/animal_mal/rating_complete.csv", encoding="latin-1")
ratings

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9
...,...,...,...
57633273,353404,502,8
57633274,353404,987,4
57633275,353404,225,8
57633276,353404,243,7


In [5]:
animes["English name"] = animes["English name"].replace("Unknown", np.NaN)
animes["English name"].fillna(animes.Name, inplace=True)

In [6]:
ratings["user_id"].value_counts()

189037    15455
162615    14864
68042     13462
283786    12778
259790     9996
          ...  
201316        1
201300        1
97555         1
201293        1
137466        1
Name: user_id, Length: 310059, dtype: int64

In [30]:
x = ratings["user_id"].value_counts() > 3000

In [31]:
x

189037     True
162615     True
68042      True
283786     True
259790     True
          ...  
117915    False
261314    False
267876    False
245012    False
75519     False
Name: user_id, Length: 562, dtype: bool

In [32]:
y = x[x].index
y.shape

(167,)

In [33]:
ratings = ratings[ratings["user_id"].isin(y)]

In [34]:
ratings

,user_id,anime_id,rating
119020,781,10122,6
119021,781,851,10
119022,781,15005,8
119023,781,34955,5
119024,781,20603,7
...,...,...,...
57381639,351801,8792,4
57381640,351801,3482,5
57381641,351801,22177,2
57381642,351801,1495,6


In [35]:
animes.rename(columns={"MAL_ID": "anime_id"}, inplace=True)

In [36]:
animes_ratings = ratings.merge(animes, on="anime_id")
animes_ratings.head()

,user_id,anime_id,rating,Name,Genres,English name,Type,Rating
0,781,10122,6,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages
1,4773,10122,6,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages
2,5648,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages
3,10255,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages
4,18355,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages


In [37]:
animes_ratings_times = animes_ratings.groupby("anime_id")["rating"].count().reset_index()

In [38]:
animes_ratings_times

,anime_id,rating
0,1,147
1,5,139
2,6,137
3,7,86
4,8,45
...,...,...
16800,47614,11
16801,47616,38
16802,47618,10
16803,48177,13


In [39]:
animes_ratings_times.rename(columns={"rating": "rating_count"}, inplace=True)
animes_ratings_times

,anime_id,rating_count
0,1,147
1,5,139
2,6,137
3,7,86
4,8,45
...,...,...
16800,47614,11
16801,47616,38
16802,47618,10
16803,48177,13


In [40]:
f_ratings = animes_ratings.merge(animes_ratings_times, on="anime_id")
f_ratings

,user_id,anime_id,rating,Name,Genres,English name,Type,Rating,rating_count
0,781,10122,6,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
1,4773,10122,6,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
2,5648,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
3,10255,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
4,18355,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
...,...,...,...,...,...,...,...,...,...
752845,308285,42476,8,Detective Conan: The Burning Galactic Railway,"Adventure, Mystery, Kids",Detective Conan: The Burning Galactic Railway,Movie,PG-13 - Teens 13 or older,1
752846,308285,42136,7,Pokemon Diamond & Pearl Atsumare! Pokemon Plan...,"Action, Adventure, Comedy, Kids, Fantasy",Pokemon Diamond & Pearl Atsumare! Pokemon Plan...,Movie,PG - Children,1
752847,328195,41635,3,Zhu Zhu Xia: Jing Su Xiao Yingxiong,"Sci-Fi, Cars, Space, Super Power, Kids",GG Bond Season 16:Racing,TV,G - All Ages,1
752848,328195,42366,6,Yasai no Yousei: N.Y. Salad The Movie,"Kids, Fantasy",Vegetable Fairies N.Y. SALAD Movie,Movie,G - All Ages,1


In [41]:
f_ratings.shape

(752850, 9)

In [43]:
final_ratings = f_ratings[f_ratings["rating_count"] > 50]
final_ratings.shape

(493980, 9)

In [44]:
final_ratings

,user_id,anime_id,rating,Name,Genres,English name,Type,Rating,rating_count
0,781,10122,6,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
1,4773,10122,6,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
2,5648,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
3,10255,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
4,18355,10122,7,Kowarekake no Orgel Special,"Sci-Fi, Slice of Life, Drama, Seinen",Kowarekake no Orgel Special,Special,G - All Ages,90
...,...,...,...,...,...,...,...,...,...
729544,335278,5784,8,Ai no Kusabi (2012),"Drama, Romance, Sci-Fi, Yaoi",Ai no Kusabi (2012),OVA,R+ - Mild Nudity,57
729545,336750,5784,8,Ai no Kusabi (2012),"Drama, Romance, Sci-Fi, Yaoi",Ai no Kusabi (2012),OVA,R+ - Mild Nudity,57
729546,341571,5784,7,Ai no Kusabi (2012),"Drama, Romance, Sci-Fi, Yaoi",Ai no Kusabi (2012),OVA,R+ - Mild Nudity,57
729547,345498,5784,5,Ai no Kusabi (2012),"Drama, Romance, Sci-Fi, Yaoi",Ai no Kusabi (2012),OVA,R+ - Mild Nudity,57


In [45]:
animes_transposition = final_ratings.pivot_table(columns="user_id", index="anime_id", values="rating")

In [46]:
animes_transposition

user_id,781,4773,5648,7179,10255,10367,11249,15083,15609,18355,...,336459,336750,337637,341571,341719,345498,348776,348906,351361,351801
anime_id,,,,,,,,,,,,,,,,,,,,,
1,7.0,8.0,8.0,10.0,10.0,9.0,10.0,10.0,10.0,8.0,...,10.0,10.0,6.0,9.0,7.0,9.0,9.0,9.0,8.0,6.0
5,NaN,8.0,8.0,7.0,10.0,8.0,8.0,8.0,10.0,7.0,...,8.0,8.0,5.0,9.0,7.0,9.0,9.0,8.0,8.0,7.0
6,10.0,8.0,8.0,9.0,8.0,6.0,10.0,8.0,10.0,7.0,...,7.0,7.0,NaN,8.0,5.0,9.0,10.0,8.0,5.0,5.0
7,10.0,7.0,8.0,7.0,8.0,NaN,9.0,7.0,NaN,NaN,...,7.0,6.0,NaN,NaN,7.0,6.0,NaN,NaN,10.0,6.0
15,NaN,NaN,NaN,NaN,8.0,NaN,8.0,NaN,9.0,7.0,...,NaN,5.0,NaN,8.0,8.0,5.0,8.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41930,7.0,8.0,NaN,5.0,NaN,4.0,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,4.0,NaN
42260,NaN,1.0,NaN,NaN,6.0,2.0,3.0,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42603,7.0,6.0,NaN,5.0,4.0,NaN,NaN,7.0,NaN,7.0,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
animes_transposition.fillna(0, inplace=True)
animes_transposition

user_id,781,4773,5648,7179,10255,10367,11249,15083,15609,18355,...,336459,336750,337637,341571,341719,345498,348776,348906,351361,351801
anime_id,,,,,,,,,,,,,,,,,,,,,
1,7.0,8.0,8.0,10.0,10.0,9.0,10.0,10.0,10.0,8.0,...,10.0,10.0,6.0,9.0,7.0,9.0,9.0,9.0,8.0,6.0
5,0.0,8.0,8.0,7.0,10.0,8.0,8.0,8.0,10.0,7.0,...,8.0,8.0,5.0,9.0,7.0,9.0,9.0,8.0,8.0,7.0
6,10.0,8.0,8.0,9.0,8.0,6.0,10.0,8.0,10.0,7.0,...,7.0,7.0,0.0,8.0,5.0,9.0,10.0,8.0,5.0,5.0
7,10.0,7.0,8.0,7.0,8.0,0.0,9.0,7.0,0.0,0.0,...,7.0,6.0,0.0,0.0,7.0,6.0,0.0,0.0,10.0,6.0
15,0.0,0.0,0.0,0.0,8.0,0.0,8.0,0.0,9.0,7.0,...,0.0,5.0,0.0,8.0,8.0,5.0,8.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41930,7.0,8.0,0.0,5.0,0.0,4.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.0,0.0
42260,0.0,1.0,0.0,0.0,6.0,2.0,3.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42603,7.0,6.0,0.0,5.0,4.0,0.0,0.0,7.0,0.0,7.0,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
from scipy.sparse import csr_matrix
# Criando uma matriz esparsa a partir dos dados tranpostos
animes_sparse = csr_matrix(animes_transposition)

In [49]:
# Treinando o algoritmo de testes
knn_model = NearestNeighbors(algorithm="brute")
knn_model.fit(animes_sparse)

NearestNeighbors(algorithm='brute')

In [66]:
animes_transposition.to_csv("animrat.csv")

In [70]:
animes_transposition.shape

(5802, 167)

In [60]:
def get_anime_name(anime_id):
  return animes.iloc[anime_id,:]["Name"]

In [71]:
distances, suggestions = knn_model.kneighbors(animes_transposition.iloc[2, :].values.reshape(1, -1))

In [72]:
for i in range(len(suggestions)):
  print(get_anime_name(animes_transposition.index[suggestions[i]]))

6                              Hachimitsu to Clover
4106                                  Tsumiki no Ie
270                                  Divergence Eve
889     Dragon Ball Z: Saiya-jin Zetsumetsu Keikaku
4901                       Shikabane Hime: Puchitto
Name: Name, dtype: object


'Cowboy Bebop'

## Importando dataset de reviews dos usuários

# Pré-processamento de dados
